In [ ]:
import os
from skimage import exposure
import numpy as np
from PIL import ImageChops, Image
import pandas as pd
import cv2
from skimage import metrics


def ssim(imageA, imageB):
    imageA = np.array(imageA)
    imageA = cv2.cvtColor(imageA, cv2.COLOR_RGB2BGR)
    imageB = np.array(imageB)
    imageB = cv2.cvtColor(imageB, cv2.COLOR_RGB2BGR)
    return metrics.structural_similarity(imageA, imageB, full=True, multichannel=True)[0]

def detect_edges(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurredmetrics.structural_similarity, 100)
    return edges

def detect_edges(image):
    # Convertir la imagen PIL a un array de numpy
    image = np.array(image)
    
    # Convertir la imagen a escala de grises
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Aplicar un suavizado Gaussiano
    blurred = cv2.GaussianBlur(gray, (7, 7), 0)
    
    # Detectar los bordes usando el algoritmo de Canny
    edges = cv2.Canny(blurred, 10, 50)
    
    return edges

def absolute_difference(imageA, imageB):
    return np.sum(np.abs(np.asarray(imageA)- np.asarray(imageB)))

def compare_histograms(imageA, imageB):
    histA = exposure.histogram(np.asarray(imageA))[0]
    histB = exposure.histogram(np.asarray(imageB))[0]
    
    # Normalizar los histogramas
    histA = histA / np.sum(histA)
    histB = histB / np.sum(histB)
    
    # Asegurarse de que ambos histogramas tengan la misma longitud
    if len(histA) > len(histB):
        histB = np.pad(histB, (0, len(histA) - len(histB)))
    elif len(histB) > len(histA):
        histA = np.pad(histA, (0, len(histB) - len(histA)))
    
    return np.sum((histA - histB) ** 2)

def image_difference(imageA, imageB):
    diff = ImageChops.difference(imageA, imageB)
    return np.sum(np.array(diff))


def image_grid(rows, cols, pre_pred):
    """
    Creates a grid of images from a list of PIL images.

    Args:
        rows (int): Number of rows in the grid.
        cols (int): Number of columns in the grid.

    Returns:
        PIL.Image: The final image grid.
    """
    list_abs = []
    list_histo = []
    list_difs = []
    list_edges = []
    list_struc = []
    w, h = (512,512)
    grid = Image.new('RGB', size=(cols * w, rows * h))
    
    imgs_i = os.listdir("Inputs")
    imgs_o = os.listdir("Outputs")
    imgs_r = os.listdir("Preds")
    
    for i, img in enumerate(imgs_i):
        a = Image.open('Outputs/' + img).resize((w,h))
        b = Image.open('Preds/'+pre_pred+ img,)
        grid.paste(Image.open('Inputs/'+ img,), box=(176, (i*h)+ 176))
        grid.paste(a, box=(512, 512*i))
        grid.paste(b, box=(1024, 512*i))
        
        list_struc.append(ssim(a,b))
        list_abs.append(absolute_difference(a,b))
        list_histo.append(compare_histograms(a,b))
        list_difs.append(image_difference(a,b))
        list_edges.append(absolute_difference(detect_edges(a),detect_edges(b)))
    
    d = pd.DataFrame({'absolute_difference':list_abs,
                     'compare_histograms':list_histo,
                     'image_difference':list_difs,
                     'detect_edges':list_edges,
                     'structural_difference':list_struc})
        
    return grid, d